<a href="https://colab.research.google.com/github/lhy-1213/skills-introduction-to-github/blob/main/HW02/HW02-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework 2-2 Hessian Matrix**

* Slides: https://speech.ee.ntu.edu.tw/~hylee/ml/ml2021-course-data/hw/HW02/HW02.pdf
* Video (Chinese): https://youtu.be/PdjXnQbu2zo
* Video (English): https://youtu.be/ESRr-VCykBs


## Hessian Matrix
Imagine we are training a neural network and we are trying to find out whether the model is at **local minima like, saddle point, or none of the above**. We can make our decision by calculating the Hessian matrix.

In practice, it is really hard to find a point where the gradient equals zero or all of the eigenvalues in Hessian matrix are greater than zero. In this homework, we make the following two assumptions:
1. View gradient norm less than 1e-3 as **gradient equals to zero**.
2. If minimum ratio is greater than 0.5 and gradient norm is less than 1e-3, then we assume that the model is at “local minima like”.

> Minimum ratio is defined as the proportion of positive eigenvalues.

## IMPORTANT NOTICE
In this homework, students with different student IDs will get different answers. Make sure to fill in your `student_id` in the following block correctly. Otherwise, your code may not run correctly and you will get a wrong answer.

In [4]:
student_id = '22024210105' # fill with your student ID

assert student_id != 'your_student_id', 'Please fill in your student_id before you start.'

## Calculate Hessian Matrix
The computation of Hessian is done by TA, you don't need to and shouldn't change the following code. The only thing you need to do is to run the following blocks and determine whether the model is at `local minima like`, `saddle point`, or `none of the above` according to the value of `gradient norm` and `minimum ratio`.

### Install Package to Compute Hessian.

The autograd-lib library is used to compute Hessian matrix. You can check the full document here https://github.com/cybertronai/autograd-lib.

In [5]:
!pip install autograd-lib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.3/857.3 kB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 71.8 MB/s eta 0:00:00


### Import Libraries


In [6]:
import numpy as np
from math import pi
from collections import defaultdict
from autograd_lib import autograd_lib

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

import warnings
warnings.filterwarnings("ignore")

### Define NN Model
The NN model here is used to fit a single variable math function.
$$f(x) = \frac{\sin(5\pi x)}{5\pi x}.$$

In [7]:
class MathRegressor(nn.Module):
    def __init__(self, num_hidden=128):
        super().__init__()
        self.regressor = nn.Sequential(
            nn.Linear(1, num_hidden),
            nn.ReLU(),
            nn.Linear(num_hidden, 1)
        )

    def forward(self, x):
        x = self.regressor(x)
        return x

### Get Pretrained Checkpoints
The pretrained checkpoints is done by TA. Each student will get a different checkpoint.

In [8]:
!gdown --id 1ym6G7KKNkbsqSnMmnxdQKHO1JBoF0LPR

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:139: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1ym6G7KKNkbsqSnMmnxdQKHO1JBoF0LPR
To: /content/data.pth
100% 34.5k/34.5k [00:00<00:00, 73.3MB/s]


### Load Pretrained Checkpoints and Training Data

In [9]:
# find the key from student_id
import re

key = student_id[-1]
if re.match('[0-9]', key) is not None:
    key = int(key)
else:
    key = ord(key) % 10

In [10]:
# load checkpoint and data corresponding to the key
model = MathRegressor()
autograd_lib.register(model)

data = torch.load('data.pth')[key]
model.load_state_dict(data['model'])
train, target = data['data']

### Function to compute gradient norm

In [11]:
# function to compute gradient norm
def compute_gradient_norm(model, criterion, train, target):
  # 把模型设置为训练模式的作用是因为有些层在训练和评估模式中不同，为了保证梯度计算的一致性，设置不同的模式
    model.train()
  # 清空模型的所有梯度，防止梯度累计
    model.zero_grad()
    # 前向传播，计算模型输出
    output = model(train)
    # criterion: 损失函数，如 nn.MSELoss()、nn.CrossEntropyLoss()等
    loss = criterion(output, target)
    loss.backward()
# 收集所有曾的梯度范数
    grads = []
# 遍历模型的回归部分的子模块：
    for p in model.regressor.children():
      # 注意下面的意思是只处理线性层
        if isinstance(p, nn.Linear):
          #计算权重梯度的L2范数，p.weight.grad: 权重的梯度张量，.norm(2): 计算L2范数（即所有元素的平方和再开方）
          # .item(): 将单元素张量转换为Python标量
            param_norm = p.weight.grad.norm(2).item()
            grads.append(param_norm)

    grad_mean = np.mean(grads) # compute mean of gradient norms

    return grad_mean

### Function to compute minimum ratio

In [12]:
# source code from the official document https://github.com/cybertronai/autograd-lib

# helper function to save activations
def save_activations(layer, A, _):
    '''
    A is the input of the layer, we use batch size of 6 here
    layer 1: A has size of (6, 1)
    layer 2: A has size of (6, 128)
    '''
    activations[layer] = A

# helper function to compute Hessian matrix
def compute_hess(layer, _, B):
    '''
    B is the backprop value of the layer
    layer 1: B has size of (6, 128)
    layer 2: B ahs size of (6, 1)
    '''
    A = activations[layer]
    BA = torch.einsum('nl,ni->nli', B, A) # do batch-wise outer product

    # full Hessian
    hess[layer] += torch.einsum('nli,nkj->likj', BA, BA) # do batch-wise outer product, then sum over the batch

In [16]:
import torch
import numpy as np
from collections import defaultdict

# function to compute the minimum ratio
# 下面函数的功能：计算神经网络Hessian矩阵正特征值的比例（最小比例），用于分析损失函数曲面的凸性，帮助诊断优化难度
def compute_minimum_ratio(model, criterion, train, target):
    # 1. 清空模型的所有梯度
    model.zero_grad()
    # 重要：确保不会与之前计算的梯度混合

    # 2. 计算Hessian矩阵的第一步：保存激活值
    # 使用autograd_lib库的钩子机制
    with autograd_lib.module_hook(save_activations):
        # 前向传播
        output = model(train)
        # 计算损失
        loss = criterion(output, target)
    # 注意：这里loss没有被使用，但计算loss会构建计算图
    # save_activations函数会在每个模块前向传播时被调用

    # 3. 计算Hessian矩阵的第二步：计算Hessian
    with autograd_lib.module_hook(compute_hess):
        # backward_hessian会使用上一步保存的激活值计算Hessian
        autograd_lib.backward_hessian(output, loss='LeastSquares')
        # 'LeastSquares'指定使用最小二乘损失计算Hessian
        # 对于MSE损失，Hessian计算更简单

    # 4. 获取所有层的Hessian矩阵
    layer_hess = list(hess.values())
    # 假设hess是一个全局字典，存储了每个层的Hessian矩阵
    # 格式：{layer_name: hessian_matrix}

    minimum_ratio = []  # 存储每层的正特征值比例

    # 5. 对每一层的Hessian矩阵进行分析
    for h in layer_hess:
        # 5.1 将Hessian矩阵重塑为方阵
        # 假设h的原始形状是(m, n)，需要重塑为(size, size)
        size = h.shape[0] * h.shape[1]
        h = h.reshape(size, size)
        # 现在h是(size, size)的方阵

        # 5.2 计算实对称矩阵的特征值
        # ✅ 修复：使用torch.linalg.eigvalsh()替代已移除的torch.symeig()
        h_eig = torch.linalg.eigvalsh(h)
        # eigvalsh()专门用于实对称矩阵，返回实数特征值
        # Hessian矩阵总是对称的（如果函数二阶连续可导）

        # 5.3 统计正特征值的数量
        num_greater = torch.sum(h_eig > 0).item()
        # h_eig > 0: 布尔张量，True表示特征值>0
        # torch.sum(): 统计True的数量
        # .item(): 转换为Python标量

        # 5.4 计算正特征值的比例
        minimum_ratio.append(num_greater / len(h_eig))
        # 比例 = 正特征值数量 / 总特征值数量

    # 6. 计算所有层比例的平均值
    ratio_mean = np.mean(minimum_ratio)  # compute mean of minimum ratio

    return ratio_mean

### Mathematical Derivation

Method used here: https://en.wikipedia.org/wiki/Gauss–Newton_algorithm

> **Notations** \\
> $\mathbf{A}$: the input of the layer. \\
> $\mathbf{B}$: the backprop value. \\
> $\mathbf{Z}$: the output of the layer. \\
> $L$: the total loss, mean squared error was used here, $L=e^2$. \\
> $w$: the weight value.

Assume that the input dimension of the layer is $n$, and the output dimension of the layer is $m$.

The derivative of the loss is

\begin{align*}
    \left(\frac{\partial L}{\partial w}\right)_{nm} &= \mathbf{A}_m \mathbf{B}_n,
\end{align*}

which can be written as

\begin{align*}
    \frac{\partial L}{\partial w} &= \mathbf{B} \times \mathbf{A}.
\end{align*}

The Hessian can be derived as

\begin{align*}
    \mathbf{H}_{ij}&=\frac{\partial^2 L}{\partial w_i \partial w_j} \\
    &= \frac{\partial}{\partial w_i}\left(\frac{\partial L}{\partial w_j}\right) \\
    &= \frac{\partial}{\partial w_i}\left(\frac{2e\partial e}{\partial w_j}\right) \\
    &= 2\frac{\partial e}{\partial w_i}\frac{\partial e}{\partial w_j}+2e\frac{\partial^2 e}{\partial w_j \partial w_i}.
\end{align*}

We neglect the second-order derivative term because the term is relatively small ($e$ is small)

\begin{align*}
    \mathbf{H}_{ij}
    &\propto \frac{\partial e}{\partial w_i}\frac{\partial e}{\partial w_j},
\end{align*}

and as the error $e$ is a constant

\begin{align*}
    \mathbf{H}_{ij}
    &\propto \frac{\partial L}{\partial w_i}\frac{\partial L}{\partial w_j},
\end{align*}

then the full Hessian becomes

\begin{align*}
    \mathbf{H} &\propto (\mathbf{B}\times\mathbf{A})\times(\mathbf{B}\times\mathbf{A}).
\end{align*}


In [17]:
# the main function to compute gradient norm and minimum ratio
def main(model, train, target):
    criterion = nn.MSELoss()

    gradient_norm = compute_gradient_norm(model, criterion, train, target)
    minimum_ratio = compute_minimum_ratio(model, criterion, train, target)

    print('gradient norm: {}, minimum ratio: {}'.format(gradient_norm, minimum_ratio))

After running this block, you will get the value of `gradient norm` and `minimum ratio`. Determine whether the model is at `local minima like`, `saddle point`, or `none of the above`, and then submit your choice to NTU COOL.

In [18]:
if __name__ == '__main__':
    # fix random seed
    torch.manual_seed(0)

    # reset compute dictionaries
    activations = defaultdict(int)
    hess = defaultdict(float)

    # compute Hessian
    main(model, train, target)

gradient norm: 0.07222428917884827, minimum ratio: 0.484375
